In [32]:
import os
from Bio import Entrez
import urllib
import requests
import re
from bs4 import BeautifulSoup
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio import pairwise2

In [22]:
def get_NCBIresults(ProtAcc):
    All_Results = []
    for ProtAcc in All_Proteins:
        Results = {}
        Results['Protein_Name'] = ProtAcc
        Entrez.email = 'adrajesh@ucsd.edu'
        paramEutils = { 'usehistory':'Y' }        # Use Entrez search history to cache results
        try:
            eSearch = Entrez.esearch(db="protein", term=ProtAcc, **paramEutils)
            res = Entrez.read(eSearch)
            ProtID = res['IdList'][0]
        except IndexError:
            print('WARNING: Protein ID unavailable for', ProtAcc)
            Results['Protein_ID'] =None
            Results['organism'] = None
            Results['genus'] =None
            Results['species'] = None
            Results['Gene_ID'] = None
            Results['Start'] = None
            Results['Assembly_Refseq_ID'] = None
            Results['Assembly_GeneBank_ID'] = None
        else:
            Results['Protein_ID'] =ProtID
            handle = Entrez.efetch(db="protein", id=ProtAcc, rettype="gp", retmode="xml")
            record =Entrez.read(handle)
            handle.close()
            Results['organism'] = record[0]['GBSeq_organism']
            Results['genus']=record[0]['GBSeq_organism'].split(' ')[0]
            try:
                eSearch2 = Entrez.esearch(db="gene", term=ProtAcc, **paramEutils)
                res2 = Entrez.read(eSearch2)
                GeneID = res2['IdList'][0]
            except IndexError:
                handle = Entrez.efetch(db="protein", id=ProtID, rettype="gp", retmode="text")
                record =handle.read()
                handle.close()
                g = record[record.find("GeneID")::]
                GeneID = g[g.find(":")+1:g.find('\"')]
                if GeneID == '':
                    print('WARNING: Gene ID unavailable for', ProtAcc)
                    Results['Gene_ID'] = None
                    Results['organism'] = None
                    Results['genus'] =None
                    Results['Start'] = None
                    Results['Assembly_Refseq_ID'] = None
                    Results['Assembly_GeneBank_ID'] = None
                else:
                    handle = Entrez.efetch(db="gene", id=GeneID, rettype="gb", retmode="text")
                    record =handle.read()
                    handle.close()
                    Ann = record[record.find("\nAnnotation")+1:record.find(")\n")]
                    AID = Ann[Ann.find("N"):Ann.find(".")+2]
                    #print(AID)
                    Start = int(Ann[Ann.find("(")+1:Ann.find("..")])
                    Results['Gene_ID'] = GeneID
                    Results['Start'] =Start
                    Results['Assembly_Refseq_ID'] = AID
                    handle = Entrez.efetch(db="nucleotide", id=AID, rettype="gb", retmode="xml")
                    record = Entrez.read(handle)
                    handle.close()
                    gbID = record[0]['GBSeq_contig']
                    gbID = gbID[gbID.find("(")+1:gbID.find(":")]
                    Results['Assembly_GeneBank_ID'] = gbID
                    handle = Entrez.efetch(db="nucleotide", rettype="gb", retmode="xml", id=gbID)
                    record = Entrez.read(handle)
                    handle.close()
                    genome = record[0]['GBSeq_sequence']
                    Results['Sequence'] = genome
            else:
                handle = Entrez.efetch(db="gene", id=GeneID, rettype="gb", retmode="text")
                record =handle.read()
                handle.close()
                Ann = record[record.find("\nAnnotation")+1:record.find(")\n")]
                AID = Ann[Ann.find("N"):Ann.find(".")+2]
                Start = int(Ann[Ann.find("(")+1:Ann.find("..")])
                Results['Gene_ID'] = GeneID
                Results['Start'] =Start
                Results['Assembly_Refseq_ID'] = AID
                handle = Entrez.efetch(db="nucleotide", id=AID, rettype="gb", retmode="xml")
                record = Entrez.read(handle)
                handle.close()
                gbID = record[0]['GBSeq_contig']
                gbID = gbID[gbID.find("(")+1:gbID.find(":")]
                Results['Assembly_GeneBank_ID'] = gbID
                handle = Entrez.efetch(db="nucleotide",id=gbID, rettype="fasta", retmode="xml")
                record =Entrez.read(handle)
                handle.close()
                genome = record[0]['TSeq_sequence']
               # genome = record[0]['GBSeq_sequence']
                Results['Sequence'] = genome
        All_Results.append(Results)
    return All_Results

In [27]:
def get_PHASTERresults(Results):
    for Result in Results:
        if Result['Gene_ID']!=None:
            RID = Result['Assembly_Refseq_ID']
            start = Result['Start']
            flag=0
            header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/32.0',}
            url = "http://phaster.ca/phaster_api?acc="+RID
            r  = requests.get(url, headers=header)
            data = r.text
            all_matches = re.findall(r'\d{1,15}-\d{1,15}',data)
            for match in all_matches:
                positions = match.split("-")
                if int(positions[0])<=start and int(positions[1])>=start:
                    PhageROI = [int(positions[0]),int(positions[1])]
                    print('found_region for',RID)
                    Result['Phage_region'] = PhageROI
                    print(Result['Phage_region'])
                    flag = 1
            if flag ==0:
                print('something is wrong, No results found for', Result['Protein_Name'],RID)

In [17]:
#search = All_Results[0]['Sequence'][1657836: 1716259]

In [141]:
#result_handle = NCBIWWW.qblast("blastn", "nt", search, entrez_query  = 'Streptococcus[organism]',hitlist_size=10,descriptions=10, alignments = 10)

In [142]:
#with open("my_blast2.xml", "w") as out_handle:
#     out_handle.write(result_handle.read())
#result_handle.close()

In [43]:
#result_handle = open("my_blast.xml")

In [31]:
#res = result_handle.read()

In [52]:
#result_handle = open("my_blast.xml")
#blast_record = NCBIXML.read(result_handle)

In [213]:
#blast_record.__dict__.keys()

dict_keys(['application', 'version', 'date', 'reference', 'query', 'query_letters', 'database', 'database_sequences', 'database_letters', 'database_name', 'posted_date', 'num_letters_in_database', 'num_sequences_in_database', 'ka_params', 'gapped', 'ka_params_gap', 'matrix', 'gap_penalties', 'sc_match', 'sc_mismatch', 'num_hits', 'num_sequences', 'num_good_extends', 'num_seqs_better_e', 'hsps_no_gap', 'hsps_prelim_gapped', 'hsps_prelim_gapped_attemped', 'hsps_gapped', 'query_id', 'query_length', 'database_length', 'effective_hsp_length', 'effective_query_length', 'effective_database_length', 'effective_search_space', 'effective_search_space_used', 'frameshift', 'threshold', 'window_size', 'dropoff_1st_pass', 'gap_x_dropoff', 'gap_x_dropoff_final', 'gap_trigger', 'blast_cutoff', 'descriptions', 'alignments', 'multiple_alignment', 'filter', 'expect'])

In [219]:
#blast_record.alignments[0].__dict__.keys()

dict_keys(['title', 'hit_id', 'hit_def', 'length', 'hsps', 'accession'])

In [220]:
#blast_record.alignments[0].hsps[0].__dict__.keys()

dict_keys(['score', 'bits', 'expect', 'num_alignments', 'identities', 'positives', 'gaps', 'align_length', 'strand', 'frame', 'query', 'query_start', 'query_end', 'match', 'sbjct', 'sbjct_start', 'sbjct_end'])

In [10]:
def getAllAlignments(Result,blastfile):
    query_start = Result['Phage_region'][0]-10000
    query_end = Result['Phage_region'][1] +10000
    if query_end>=len(Result['Sequence']):
        print("warning, query to exceeds query length, clipping")
    search = Result['Sequence'][query_start:query_end]
    result_handle = NCBIWWW.qblast("blastn", "nt", search, entrez_query  = Result['genus']+'[organism]',hitlist_size=10)
    with open(blastfile, "w") as out_handle:
        out_handle.write(result_handle.read())
    result_handle.close()

In [11]:
def getOptAlignment(blastfile):
    result_handle = open(blastfile)
    record = NCBIXML.read(result_handle)
    result_handle.close()
    for align in record.alignments:
        total_query = 0
        for i in range(4):
            total_query+= align.hsps[i].align_length
            #print(total_query)
        total_query_range = total_query/record.query_length
        print(total_query_range)
        if total_query_range<=0.6:
            Opt_alignment = align
            print(Opt_alignment.title)
            return Opt_alignment

In [12]:
def getAtt(Alignment):
    return_pair =[]
    all_start_ends =[]
    for hsp in Alignment.hsps:
        start_end = {}
        start_end['subject_start'] = hsp.sbjct_start
        start_end['subject_end'] = hsp.sbjct_end
        start_end['query_start'] = hsp.query_start
        start_end['query_end'] = hsp.query_end
        start_end['expect'] =hsp.expect
        start_end['query'] = hsp.query
        start_end['subject'] = hsp.sbjct
        all_start_ends.append(start_end)
        sorted_start_ends = sorted(all_start_ends, key=lambda k: k['subject_start']) 
    while(1):
        del_indices = []
        for i in range(len(sorted_start_ends)-1):
            if sorted_start_ends[i]['subject_start']< sorted_start_ends[i+1]['subject_start'] and sorted_start_ends[i]['subject_end']>sorted_start_ends[i+1]['subject_end']:
                del_indices.append(i+1)
    #    print(del_indices)
        for index in sorted(del_indices, reverse=True):
            del sorted_start_ends[index]
        if len(del_indices) ==0:
            break
    possible_overlaps = []
    c=0
    for i in range(len(sorted_start_ends)-1):        
        if sorted_start_ends[i]['subject_end']> sorted_start_ends[i+1]['subject_start'] and sorted_start_ends[i]['subject_end']<sorted_start_ends[i+1]['subject_end']:
            if sorted_start_ends[i]['expect'] ==0 and sorted_start_ends[i+1]['expect'] == 0 and sorted_start_ends[i]['subject_end']-sorted_start_ends[i+1]['subject_start']<40 and sorted_start_ends[i+1]['query_start']-sorted_start_ends[i]['query_start']>2000:
                return_pair.append(sorted_start_ends[i])
                return_pair.append(sorted_start_ends[i+1])
                print('\nquery_start difference', sorted_start_ends[i+1]['query_start']-sorted_start_ends[i]['query_start'])
                c = c+1
                if c>1:
                    print('warning: more than one subject overlap region')
    return return_pair

In [13]:
def get_index(alignment):
    matches = []
    breaks =[]
    all_breaks =[]
    overlaps = []
    breaks.append(0)
    for i, (a, b) in enumerate(zip(alignment[0], alignment[1])):
        if a == b:
            matches.append(i)
            
    for i in range(len(matches)-1):
        if matches[i+1]-matches[i]>1:
            breaks.append(matches[i])
            breaks.append(matches[i+1])
    breaks.append(len(alignment[0])-1)
    
    for i in range(0,len(breaks),2):
        if breaks[i]!= breaks[i+1]:
            all_breaks.append((breaks[i],breaks[i+1]))
    for br in all_breaks:
        overlaps.append(alignment[0][br[0]:br[1]+1])
    return all_breaks,overlaps

In [35]:
def getattsites(optalign,search):
    potential_attB_sites = []
    potential_attP_sites = []
    pair1 = getAtt(optalign)
    if(len(pair1) ==0):
        print('No Site Found')
        return None,None,None,None,None
    align_length = pair1[0]['subject_end']-pair1[1]['subject_start']+1
    X = pair1[0]['query'][-align_length::].upper()
    Y = pair1[1]['query'][0:align_length].upper()
    alignments = pairwise2.align.localms(X,Y,3,-1,-3,-2)

    # Use format_alignment method to format the alignments in the list
    print('query overlap aligned:')
    for i in alignments:
        print(pairwise2.format_alignment(*i))
    range_1_sbjct_end = pair1[0]['subject_end']
    range_1_query_end = pair1[0]['query_end']
    range_2_sbjct_start = pair1[1]['subject_start']
    range_2_query_start = pair1[1]['query_start']
    overlap_size = range_1_sbjct_end - range_2_sbjct_start + 1
    attL_start = range_1_query_end - overlap_size - 25
    attL_stop = range_1_query_end + 25
    attR_start = range_2_query_start - 25-1
    attR_stop = range_2_query_start + overlap_size + 25-1
    attL_seq = search[attL_start:attL_stop].upper()
    attR_seq = search[attR_start:attR_stop].upper()
    reference_attB = pair1[0]['subject'][-overlap_size-25::]+pair1[1]['subject'][overlap_size:overlap_size+25]
    reference_attB = reference_attB.upper()
    print('reference attB: ',reference_attB)
    print('attL: ',attL_seq)
    print('attR: ',attR_seq)
    inds,overlaps = get_index(alignments[0])
    print('\npossible overlaps: ',overlaps)
    for ind,overlap in zip(inds,overlaps):
        if len(overlap)<= 18:
            attB = attL_seq[0:25+ind[0]][-25::]+overlap+attR_seq[25+ind[1]+1::][0:25]
            attP = attR_seq[0:25+ind[0]][-25::]+overlap+attL_seq[25+ind[1]+1::][0:25]
            potential_attB_sites.append(attB)
            potential_attP_sites.append(attP)
        else:
            print('overlap more than 18bp, ignoring.')
    max_score =0  
    print('potential attB sites:',potential_attB_sites)
    print('\nchecking possible attB sites:\n')
    for i,attBsite in enumerate(potential_attB_sites):
        alignments = pairwise2.align.localms(attBsite,reference_attB,1,0,-3,-2)
        score = alignments[0][2] - len(overlaps[i])
        if score>max_score:
            max_score = score
            correct_index = i
        # Use format_alignment method to format the alignments in the list
        for i in alignments:
            print(pairwise2.format_alignment(*i))
        print("Corrected score=",score)
        print('\n')
    if max_score>0:
        print('Best Regions:\n')
        print('attB: ',potential_attB_sites[correct_index])
        print('attP: ',potential_attP_sites[correct_index])
        print('overlap: ',overlaps[correct_index])
        return overlaps[correct_index],potential_attB_sites[correct_index], potential_attP_sites[correct_index],attL_seq,attR_seq
    else:
        print('all scores<0')
        return None,None,None,None,None

In [15]:
def getBlastResults(Results):
    for Result in Results:
        if 'Phage_region'in Result:
            print('\n\n\n******',Result['organism'])
            print(Result['Protein_Name'])
            if not os.path.isfile('BlastFiles/'+Result['Protein_Name']+'.xml'):
                getAllAlignments(Result, 'BlastFiles/'+Result['Protein_Name']+'.xml')
            optalign =getOptAlignment('BlastFiles/'+Result['Protein_Name']+'.xml')
            if optalign!= None:
                search = Result['Sequence'][Result['Phage_region'][0]-10000:Result['Phage_region'][1] +10000]
                o,b,p,l,r = getattsites(optalign,search)
                if o!=None:
                    Result['overlap'] = o
                    Result['attB_site']=b
                    Result['attP_site']=p
                    Result['attL_site']=l
                    Result['attR_site']=r

In [16]:
All_Proteins = ['YP_353073','CBG73463','NP_268897','YP_002747001','BAF03598','BAG46462','YP_003251752','BAE05705','BAF67264','YP_003880342','YP_001886479','YP_005759947','YP_001376196','NP_470568','YP_006685721','YP_006538656','YP_189066','YP_002736920','FM864213','YP_006082695','YP_003445547','YP_004586821','YP_001089468','YP_005679179','YP_001384783','YP_001392519','YP_005869510','YP_001271396','YP_001646422','YP_002336631','YP_005549228','YP_706485','YP_002804732','YP_003472505']

In [24]:
All_Results = get_NCBIresults(All_Proteins)

In [28]:
get_PHASTERresults(All_Results)

found_region for NC_007493.2
[1667836, 1706259]
something is wrong, No results found for CBG73463 NC_013929.1
found_region for NC_002737.2
[526678, 570489]
found_region for NC_012471.1
[1729455, 1775766]
found_region for NC_010805.1
[1616527, 1665330]
found_region for NC_013411.1
[594415, 626069]
found_region for NC_007168.1
[2352533, 2397948]
found_region for NC_009641.1
[1098794, 1162158]
found_region for NC_014498.1
[1989782, 2034139]
found_region for NC_010674.1
[2326784, 2362434]
found_region for NC_017353.1
[887270, 936704]
found_region for NC_009674.1
[2983420, 3021377]
found_region for NC_003212.1
[1245851, 1297054]
found_region for NC_018588.1
[2381577, 2420764]
found_region for NC_018221.1
[2310353, 2378090]
found_region for NC_002976.3
[1557651, 1638109]
found_region for NC_012466.1
[1737927, 1786714]
found_region for NC_017621.1
[1166675, 1203771]
found_region for NC_013853.1
[390924, 446969]
found_region for NC_015660.1
[735196, 791797]
found_region for NC_009089.1
[336704

In [36]:
getBlastResults(All_Results[0:13])




****** Rhodobacter sphaeroides 2.4.1
YP_353073
1.0144121322082056
1.0144121322082056
1.0144292487547713
1.0144121322082056
1.0144292487547713
0.5710764596135084
gi|1523754079|gb|CP033434.1| Rhodobacter sphaeroides strain AB24 chromosome 1

query_start difference 48470
query overlap aligned:
AAGAATGTGAAGGG
|||||..|||||||
AAGAAGATGAAGGG
  Score=34

reference attB:  GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTCCGGGCCGAG
attL:  GAACTCCGCCGGGCCCATCTGGTCGAAGAATGTGAAGGGTATTTTACCCTTGTCGTTTCAGTGA
attR:  TGGGGTCACAATACCAATCATGTTCAAGAAGATGAAGGGGCCCACCATCTGCCTCCGGGCCGAG

possible overlaps:  ['AAGAA', 'TGAAGGG']
potential attB sites: ['GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTC', 'GCCGGGCCCATCTGGTCGAAGAATGTGAAGGGGCCCACCATCTGCCTCCGGGCCGAG']

checking possible attB sites:

1 GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTC
  |||||||||||||||||||||||||||||||||||||||||||||||||||||||
1 GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTC
  Score=55

Corrected score= 50

In [37]:
getBlastResults(All_Results[14::])




****** Listeria monocytogenes SLCC2372
YP_006685721
1.0622265024414146
1.0622265024414146
0.8835555105006166
0.9337523442647879
0.8207376619865849
1.020426782908409
0.9851487657762684
0.729366245966175
0.729366245966175
0.729366245966175



****** Enterococcus faecalis D32
YP_006538656
1.0423994437922428
0.7702793576256312
0.7439164776548093
0.596145297878888
gi|1678198132|gb|CP030042.1| Enterococcus faecalis strain C25 chromosome, complete genome
No Site Found



****** Staphylococcus epidermidis RP62A
YP_189066
1.0457803261064325
0.8709510442174839
0.8985446654323199
0.8838917756674431
0.20579744768958172
gi|27316888|gb|AE015929.1| Staphylococcus epidermidis ATCC 12228, complete genome
No Site Found



****** Streptococcus pneumoniae JJA
YP_002736920
1.2434762382426912
0.5876255687847994
gi|1112988118|gb|CP018136.1| Streptococcus pneumoniae strain SP49, complete genome

query_start difference 12524

query_start difference 63980
query overlap aligned:
TCGCTTTCAAATT
|||||||||.|||
TC

In [540]:
getBlastResults([All_Results[0]])




****** Rhodobacter sphaeroides 2.4.1
YP_353073
1.0144121322082056
1.0144121322082056
1.0144292487547713
1.0144121322082056
1.0144292487547713
0.5710764596135084
gi|1523754079|gb|CP033434.1| Rhodobacter sphaeroides strain AB24 chromosome 1

query_start difference 48470
query overlap aligned:
AAGAATGTGAAGGG
|||||..|||||||
AAGAAGATGAAGGG
  Score=34

reference attB:  GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTCCGGGCCGAG
attL:  GAACTCCGCCGGGCCCATCTGGTCGAAGAATGTGAAGGGTATTTTACCCTTGTCGTTTCAGTGA
attR:  TGGGGTCACAATACCAATCATGTTCAAGAAGATGAAGGGGCCCACCATCTGCCTCCGGGCCGAG

possible overlaps:  ['AAGAA', 'TGAAGGG']
potential attB sites: ['GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTC', 'GCCGGGCCCATCTGGTCGAAGAATGTGAAGGGGCCCACCATCTGCCTCCGGGCCGAG']

checking possible attB sites:

1 GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTC
  |||||||||||||||||||||||||||||||||||||||||||||||||||||||
1 GAACTCCGCCGGGCCCATCTGGTCGAAGAAGATGAAGGGGCCCACCATCTGCCTC
  Score=55

Corrected score= 50